#### Task

It is necessary to analyze the impact on sales, marketing changes in three stores: 77, 86 and 88. The period of marketing activities lasted from the beginning of February 2019 to the end of April 2019.
It is necessary to check with similar stores from our network of stores.

Recommend whether you want to implement these marketing activities throughout the network.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('.\data\QVI_data.csv')
df.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [3]:
df.columns = df.columns.str.lower()

In [4]:

df.info()
df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   lylty_card_nbr    264834 non-null  int64  
 1   date              264834 non-null  object 
 2   store_nbr         264834 non-null  int64  
 3   txn_id            264834 non-null  int64  
 4   prod_nbr          264834 non-null  int64  
 5   prod_name         264834 non-null  object 
 6   prod_qty          264834 non-null  int64  
 7   tot_sales         264834 non-null  float64
 8   pack_size         264834 non-null  int64  
 9   brand             264834 non-null  object 
 10  lifestage         264834 non-null  object 
 11  premium_customer  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


,lylty_card_nbr,store_nbr,txn_id,prod_nbr,prod_qty,tot_sales,pack_size
count,2.648340e+05,264834.000000,2.648340e+05,264834.000000,264834.000000,264834.000000,264834.000000
mean,1.355488e+05,135.079423,1.351576e+05,56.583554,1.905813,7.299346,182.425512
std,8.057990e+04,76.784063,7.813292e+04,32.826444,0.343436,2.527241,64.325148
min,1.000000e+03,1.000000,1.000000e+00,1.000000,1.000000,1.500000,70.000000
25%,7.002100e+04,70.000000,6.760050e+04,28.000000,2.000000,5.400000,150.000000
50%,1.303570e+05,130.000000,1.351365e+05,56.000000,2.000000,7.400000,170.000000
75%,2.030940e+05,203.000000,2.026998e+05,85.000000,2.000000,9.200000,175.000000
max,2.373711e+06,272.000000,2.415841e+06,114.000000,5.000000,29.500000,380.000000


In [5]:
df['date'] = pd.to_datetime(df['date'])

In [6]:
df['month'] = df['date'].dt.to_period('M')

In [7]:
trial_stores = [77, 86, 88]
trial_period = ['2019-02', '2019-03', '2019-04']
trial_period = pd.PeriodIndex(trial_period, freq='M')

Create a table with metrics

In [8]:
monthly_metrics = df.groupby(['store_nbr', 'month']).agg(
    total_sales=('tot_sales', 'sum'),
    total_customers=('lylty_card_nbr', pd.Series.nunique),
    transactions=('txn_id', 'nunique')
).reset_index()

monthly_metrics['avg_transactions_per_customer'] = monthly_metrics['transactions'] / monthly_metrics['total_customers']
monthly_metrics


,store_nbr,month,total_sales,total_customers,transactions,avg_transactions_per_customer
0,1,2018-07,206.9,49,52,1.061224
1,1,2018-08,176.1,42,43,1.023810
2,1,2018-09,278.8,59,62,1.050847
3,1,2018-10,188.1,44,45,1.022727
4,1,2018-11,192.6,46,47,1.021739
...,...,...,...,...,...,...
3164,272,2019-02,395.5,45,48,1.066667
3165,272,2019-03,442.3,50,53,1.060000
3166,272,2019-04,445.1,54,55,1.018519
3167,272,2019-05,314.6,34,40,1.176471


Function for assessing the similarity between stores

In [9]:
def calculate_similarity(trial_store_id, metrics_df, metric_name='total_sales'):
    trial_data = metrics_df[metrics_df['store_nbr'] == trial_store_id].sort_values('month')
    scores = {}

    for store_id in metrics_df['store_nbr'].unique():
        if store_id == trial_store_id:
            continue

        control_data = metrics_df[metrics_df['store_nbr'] == store_id].sort_values('month')

        # Check if the lengths of the dataframes are equal
        if len(trial_data) != len(control_data):
            continue

        # pearson correlation
        correlation, _ = pearsonr(trial_data[metric_name], control_data[metric_name])

        # euclidean distance
        distance = euclidean(trial_data[metric_name], control_data[metric_name])
        scores[store_id] = {'correlation': correlation, 'distance': distance}

    # DataFrame
    similarity_df = pd.DataFrame.from_dict(scores, orient='index')
    similarity_df['scaled_distance'] = 1 - (similarity_df['distance'] - similarity_df['distance'].min()) / (similarity_df['distance'].max() - similarity_df['distance'].min())
    similarity_df['combined_score'] = (similarity_df['correlation'] + similarity_df['scaled_distance']) / 2

    return similarity_df.sort_values('combined_score', ascending=False)


In [10]:
calculate_similarity(trial_stores[0], monthly_metrics, metric_name='total_sales')

,correlation,distance,scaled_distance,combined_score
41,0.762292,152.440152,0.991912,0.877102
167,0.696075,230.361238,0.972940,0.834508
35,0.699708,419.915837,0.926787,0.813247
20,0.620701,185.252132,0.983923,0.802312
233,0.613063,157.082749,0.990782,0.801922
...,...,...,...,...
203,-0.230314,3345.789249,0.214397,-0.007959
55,-0.497079,2266.464362,0.477190,-0.009944
223,-0.497214,2380.155397,0.449509,-0.023852
4,-0.294778,3401.930964,0.200727,-0.047025


Selection of control store for pilot

In [11]:

control_matches = {}

for trial in trial_stores:
    similarity = calculate_similarity(trial, monthly_metrics, metric_name='total_sales')
    best_match = similarity.index[0]
    control_matches[trial] = best_match
    print(f"For trial store {trial} control store — {best_match}")


For trial store 77 control store — 41
For trial store 86 control store — 109
For trial store 88 control store — 201


In [12]:
control_matches

{77: np.int64(41), 86: np.int64(109), 88: np.int64(201)}

Comparison of metrics during the trial period

In [ ]:
def compare_trial_effect(trial_id, control_id, metric_name='total_sales'):
    # get data for the trial period
    trial_data = monthly_metrics[(monthly_metrics['store_nbr'] == trial_id) & 
                                 (monthly_metrics['month'].isin(trial_period))]
    control_data = monthly_metrics[(monthly_metrics['store_nbr'] == control_id) & 
                                   (monthly_metrics['month'].isin(trial_period))]
    
    
    # sample
    sample_df = trial_data.merge(control_data, on='month', suffixes=('_trial', '_control'))
    sample_df['diff_sales'] = sample_df['total_sales_trial'] - sample_df['total_sales_control']
    sample_df['diff_costomers'] = sample_df['total_customers_trial'] - sample_df['total_customers_control']
    sample_df['diff_transactions'] = sample_df['transactions_trial'] - sample_df['transactions_control']
    sample_df['diff_avg_transactions_per_customer'] = sample_df['avg_transactions_per_customer_trial'] - sample_df['avg_transactions_per_customer_control']
    sample_df['percent_change'] = (sample_df['diff_sales'] / sample_df['total_sales_control']) * 100
 
        
    return sample_df

In [ ]:
results = []
comp_df_full = pd.DataFrame()

# Get the data for every pair of stores.
for trial_id, control_id in control_matches.items():
    comp_df = compare_trial_effect(trial_id, control_id, metric_name='total_sales')
    comp_df_full = pd.concat([comp_df_full, comp_df], ignore_index=True)
    
    # Calculate metrics 
    result = comp_df.groupby('store_nbr_trial').agg(
        store_nbr_control=('store_nbr_control', 'first'),
        diff_sales=('diff_sales', 'sum'),
        diff_costomers=('diff_costomers', 'sum'),
        diff_transactions=('diff_transactions', 'sum'),
        diff_avg_transactions_per_customer=('diff_avg_transactions_per_customer', 'sum'),
        diff_sales_percent_change=('diff_sales', lambda x: (sum(x) / sum(comp_df['total_sales_control'])) * 100),
        diff_costomers_percent_change=('diff_costomers', lambda x: (sum(x) / sum(comp_df['total_customers_control'])) * 100),
        diff_transactions_percent_change=('diff_transactions', lambda x: (sum(x) / sum(comp_df['transactions_control'])) * 100),
        diff_avg_transactions_percent_change=('diff_avg_transactions_per_customer', lambda x: (sum(x) / sum(comp_df['avg_transactions_per_customer_control'])) * 100),
        
    ).reset_index()
    
    # add result to the list
    results.append(result)


# Concatenate all results into a single DataFrame
final_results = pd.concat(results, ignore_index=True)

In [15]:
comp_df_full

,store_nbr_trial,month,total_sales_trial,total_customers_trial,transactions_trial,avg_transactions_per_customer_trial,store_nbr_control,total_sales_control,total_customers_control,transactions_control,avg_transactions_per_customer_control,diff_sales,diff_costomers,diff_transactions,diff_avg_transactions_per_customer,percent_change
0,77,2019-02,235.0,45,45,1.000000,41,234.6,50,50,1.000000,0.4,-5,-5,0.000000,0.170503
1,77,2019-03,278.5,50,55,1.100000,41,226.2,46,49,1.065217,52.3,4,6,0.034783,23.121132
2,77,2019-04,263.5,47,48,1.021277,41,231.3,47,49,1.042553,32.2,0,-1,-0.021277,13.921314
3,86,2019-02,913.2,107,138,1.289720,109,858.4,91,122,1.340659,54.8,16,16,-0.050940,6.383970
4,86,2019-03,1026.8,115,140,1.217391,109,1039.2,114,145,1.271930,-12.4,1,-5,-0.054539,-1.193226
5,86,2019-04,848.2,105,126,1.200000,109,728.6,80,104,1.300000,119.6,25,22,-0.100000,16.415043
6,88,2019-02,1370.2,124,153,1.233871,201,1139.2,111,128,1.153153,231.0,13,25,0.080718,20.277388
7,88,2019-03,1477.2,134,169,1.261194,201,1364.2,130,152,1.169231,113.0,4,17,0.091963,8.283243
8,88,2019-04,1439.4,128,162,1.265625,201,1246.6,122,146,1.196721,192.8,6,16,0.068904,15.466068


In [16]:
final_results

,store_nbr_trial,store_nbr_control,diff_sales,diff_costomers,diff_transactions,diff_avg_transactions_per_customer,diff_sales_percent_change,diff_costomers_percent_change,diff_transactions_percent_change,diff_avg_transactions_percent_change
0,77,41,84.9,-1,0,0.013506,12.267013,-0.699301,0.000000,0.434588
1,86,109,162.0,42,33,-0.205478,6.168609,14.736842,8.894879,-5.251720
2,88,201,536.8,23,58,0.241585,14.314667,6.336088,13.615023,6.864949


In [17]:
final_results[['store_nbr_trial', 'store_nbr_control', 'diff_sales_percent_change', 'diff_sales', 'diff_costomers', 'diff_transactions', 'diff_avg_transactions_per_customer']]

,store_nbr_trial,store_nbr_control,diff_sales_percent_change,diff_sales,diff_costomers,diff_transactions,diff_avg_transactions_per_customer
0,77,41,12.267013,84.9,-1,0,0.013506
1,86,109,6.168609,162.0,42,33,-0.205478
2,88,201,14.314667,536.8,23,58,0.241585


#### Analysis result

If we take as a basis that changes of more than 10% are significant, we see that in store 77 and 88 sales increased significantly, and in 86 stores there was also growth, but not more than 10 percent.

I recommend implementing changes to all stores.